In [3]:
import os
import sys
from prettytable import PrettyTable

# Import the required functions from the codebase
sys.path.insert(0, os.path.abspath("../../"))
from data_processing.aggregate_logs_to_csv import load_game_log, TerminationReason

# Define the root directory where logs are stored
root_dir = "."

# Get all direct child folders
child_folders = [
    d for d in os.listdir(root_dir)
    if os.path.isdir(os.path.join(root_dir, d)) and not d.startswith(".")
]

# Initialize a dictionary to store counts for each folder
folder_counts = {}

# Process each folder
for folder in child_folders:
    folder_path = os.path.join(root_dir, folder)
    
    # Initialize counters for this folder
    reason_counts = {reason.value: 0 for reason in TerminationReason}
    total_logs = 0
    black_wins = 0
    black_losses = 0
    draws = 0
    
    # Walk through all files in the folder
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".json") and not file.endswith("_aggregate_results.json"):
                file_path = os.path.join(subdir, file)
                try:
                    log = load_game_log(file_path)
                    # Count the termination reason
                    if log.reason in reason_counts:
                        reason_counts[log.reason] += 1
                    # Count black player wins
                    if log.winner == "Player_Black" or log.winner == "NoN_Synthesizer":
                        black_wins += 1
                    # Count black player losses (white wins)
                    if log.winner == "Chess_Engine_Dragon_White":
                        black_losses += 1
                    # Count draws (NONE winner and not error)
                    if (
                        log.winner == "NONE"
                        and log.reason != TerminationReason.ERROR.value
                    ):
                        draws += 1
                    total_logs += 1
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")
    
    # Store the counts for this folder
    folder_counts[folder] = {
        "total": total_logs,
        "black_wins": black_wins,
        "black_losses": black_losses,
        "draws": draws,
        "reasons": reason_counts
    }

# Create a pretty table for output
table = PrettyTable()

# Define columns: folder name, total, error count, black win rate, win rate excl. errors, loss rate, loss rate excl. errors
table.field_names = [
    "Folder",
    "Tota logs",
    "Errors",
    "Black Win Rate (%)",
    "Win Rate Excl. Errors (%)",
    "Black Loss Rate (%)",
    "Loss Rate Excl. Errors (%)",
]

# Sort the folders alphabetically before adding to the table
for folder in sorted(folder_counts.keys()):
    counts = folder_counts[folder]
    error_count = counts["reasons"].get(TerminationReason.ERROR.value, 0)
    win_rate = (counts["black_wins"] / counts["total"] * 100) if counts["total"] > 0 else 0
    loss_rate = (counts["black_losses"] / counts["total"] * 100) if counts["total"] > 0 else 0

    # Calculate win rate and loss rate excluding error games
    non_error_games = counts["total"] - error_count
    win_rate_excl_errors = (counts["black_wins"] / non_error_games * 100) if non_error_games > 0 else 0
    loss_rate_excl_errors = (counts["black_losses"] / non_error_games * 100) if non_error_games > 0 else 0

    row = [
        folder,
        counts["total"],
        error_count,
        f"{win_rate:.2f}",
        f"{win_rate_excl_errors:.2f}",
        f"{loss_rate:.2f}",
        f"{loss_rate_excl_errors:.2f}"
    ]
    table.add_row(row)

# Print the table
print(table)

+-----------------------------------------------------------+-----------+--------+--------------------+---------------------------+---------------------+----------------------------+
|                           Folder                          | Tota logs | Errors | Black Win Rate (%) | Win Rate Excl. Errors (%) | Black Loss Rate (%) | Loss Rate Excl. Errors (%) |
+-----------------------------------------------------------+-----------+--------+--------------------+---------------------------+---------------------+----------------------------+
|     lvl-1_vs_3x-gpt-4.1-mini-2025-04-14-low_41mini-t03    |     33    |   0    |        6.06        |            6.06           |        51.52        |           51.52            |
| lvl-1_vs_claude-3-7-sonnet-20250219-thinking-budget-10000 |     33    |   0    |        0.00        |            0.00           |        66.67        |           66.67            |
|  lvl-1_vs_claude-3-7-sonnet-20250219-thinking-budget-5000 |     33    |   0    |   

<Figure size 640x480 with 0 Axes>